In [1]:
import networkx as nx
import networkx.algorithms.approximation as approx
from networkx.algorithms.tree.mst import random_spanning_tree
import tsplib95

import pulp

import itertools

problem = tsplib95.load("p43.atsp")

G = problem.get_graph()

G.is_directed() 

True

In [2]:

# only keep weights: (i,j,{'weight': weight, 'is_fixed": True}) -> G[i][j] = weight
for u, v, data in G.edges(data=True):
    weight = data.get('weight', 0)  
    G[u][v].clear()  
    G[u][v]['weight'] = weight

# making sure G is complete

for i in G.nodes:
    for j in G.nodes:
        if i != j and not G.has_edge(i, j):
            G.add_edge(i, j, weight=9999)
            print(f"[!] added edge {i} {j} with weight 9999")

G.edges(data=True)



OutEdgeDataView([(0, 0, {'weight': 0}), (0, 1, {'weight': 26}), (0, 2, {'weight': 26}), (0, 3, {'weight': 26}), (0, 4, {'weight': 10}), (0, 5, {'weight': 60}), (0, 6, {'weight': 60}), (0, 7, {'weight': 60}), (0, 8, {'weight': 68}), (0, 9, {'weight': 68}), (0, 10, {'weight': 68}), (0, 11, {'weight': 68}), (0, 12, {'weight': 92}), (0, 13, {'weight': 92}), (0, 14, {'weight': 92}), (0, 15, {'weight': 116}), (0, 16, {'weight': 116}), (0, 17, {'weight': 116}), (0, 18, {'weight': 164}), (0, 19, {'weight': 164}), (0, 20, {'weight': 164}), (0, 21, {'weight': 84}), (0, 22, {'weight': 36}), (0, 23, {'weight': 36}), (0, 24, {'weight': 42}), (0, 25, {'weight': 46}), (0, 26, {'weight': 120}), (0, 27, {'weight': 27}), (0, 28, {'weight': 27}), (0, 29, {'weight': 61}), (0, 30, {'weight': 61}), (0, 31, {'weight': 69}), (0, 32, {'weight': 69}), (0, 33, {'weight': 93}), (0, 34, {'weight': 93}), (0, 35, {'weight': 21}), (0, 36, {'weight': 27}), (0, 37, {'weight': 31}), (0, 38, {'weight': 372}), (0, 39, {'w

In [3]:
def HK(G):
    """
    Compute the Held-Karp relaxation for the ATSP following Asadpour's formulation.
    """
    if not G.is_directed():
        raise ValueError("Graph must be a directed graph (DiGraph).")

    V = list(G.nodes)
    A = list(G.edges)
    c = nx.get_edge_attributes(G, 'weight')

    prob = pulp.LpProblem("Held-Karp_Relaxation", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", A, lowBound=0, cat='Continuous')

    # objective
    prob += pulp.lpSum([c[arc] * x[arc] for arc in A]), "Total_Cost"

    # flow conservation constraints (3.3)
    for v in V:
        prob += pulp.lpSum([x[(v, w)] for w in G.successors(v)]) == 1, f"Outflow_{v}"
        prob += pulp.lpSum([x[(u, v)] for u in G.predecessors(v)]) == 1, f"Inflow_{v}"

    # subset cut constraints (3.2)
    for size in range(1, len(V)):
        for U in itertools.combinations(V, size):
            U = set(U)
            if len(U) < len(V):  
                delta_plus_U = [(u, v) for u in U for v in G.successors(u) if v not in U]
                prob += pulp.lpSum([x[arc] for arc in delta_plus_U]) >= 1, f"Cut_{U}"


    prob.solve()

    # extract x_star
    x_star = {arc: (x[arc].varValue or 0) for arc in A}

    # post-processing and scaling to z_star (3.5)
    z_star = {}
    scale_factor = (len(V) - 1) / len(V)  # (n-1)/n scaling factor from paper
    for (u, v) in A:
        frequency = x_star[(u, v)] + x_star.get((v, u), 0)
        if frequency > 0:
            z_star[(u, v)] = scale_factor * frequency

    # just some sanity checks
    support_graph = nx.Graph()
    for (u, v), val in z_star.items():
        if val > 0:
            support_graph.add_edge(u, v)
    
    if not nx.is_connected(support_graph):
        raise ValueError("Held-Karp solution produced disconnected support graph!")

    return pulp.value(prob.objective), z_star

In [4]:
HK_opt, z_star = HK(G)

print("Held-Karp relaxation optimal cost:", HK_opt)
print("Symmetrized solution:", z_star)

if isinstance(z_star, nx.DiGraph):
    print(f"z_star is digraph: {z_star}")
    for u,v in z_star.edges:
        print(u,v, G[u][v]['weight'])

else:
    print(f"z_star is dict: {z_star}")
    for u,v in z_star.keys():
        if u < v: 
            print(u, v, z_star[(u, v)])


In [ ]:
if not isinstance(z_star, dict):
    print(_shortcutting(nx.eulerian_circuit(z_star)))
    exit()

In [ ]:
def spanning_tree_distribution(G, z):
    """
    Find the asadpour exponential distribution of spanning trees.

    Solves the Maximum Entropy Convex Program in the Asadpour algorithm [1]_
    using the approach in section 7 to build an exponential distribution of
    undirected spanning trees.

    This algorithm ensures that the probability of any edge in a spanning
    tree is proportional to the sum of the probabilities of the tress
    containing that edge over the sum of the probabilities of all spanning
    trees of the graph.

    Parameters
    ----------
    G : nx.MultiGraph
        The undirected support graph for the Held Karp relaxation

    z : dict
        The output of `held_karp_ascent()`, a scaled version of the Held-Karp
        solution.

    Returns
    -------
    gamma : dict
        The probability distribution which approximately preserves the marginal
        probabilities of `z`.
    """
    from math import exp
    from math import log as ln

    def q(e):
        """
        The value of q(e) is described in the Asadpour paper is "the
        probability that edge e will be included in a spanning tree T that is
        chosen with probability proportional to exp(gamma(T))" which
        basically means that it is the total probability of the edge appearing
        across the whole distribution.

        Parameters
        ----------
        e : tuple
            The `(u, v)` tuple describing the edge we are interested in

        Returns
        -------
        float
            The probability that a spanning tree chosen according to the
            current values of gamma will include edge `e`.
        """
        # Create the laplacian matrices
        for u, v, d in G.edges(data=True):
            d[lambda_key] = exp(gamma[(u, v)])
        G_Kirchhoff = nx.total_spanning_tree_weight(G, lambda_key)
        G_e = nx.contracted_edge(G, e, self_loops=False)
        G_e_Kirchhoff = nx.total_spanning_tree_weight(G_e, lambda_key)

        # Multiply by the weight of the contracted edge since it is not included
        # in the total weight of the contracted graph.
        return exp(gamma[(e[0], e[1])]) * G_e_Kirchhoff / G_Kirchhoff

    # initialize gamma to the zero dict
    gamma = {}
    for u, v, _ in G.edges:
        gamma[(u, v)] = 0

    # set epsilon
    EPSILON = 0.2

    # pick an edge attribute name that is unlikely to be in the graph
    lambda_key = "spanning_tree_distribution's secret attribute name for lambda"

    while True:
        # We need to know that know that no values of q_e are greater than
        # (1 + epsilon) * z_e, however changing one gamma value can increase the
        # value of a different q_e, so we have to complete the for loop without
        # changing anything for the condition to be meet
        in_range_count = 0
        # Search for an edge with q_e > (1 + epsilon) * z_e
        for u, v in gamma:
            e = (u, v)
            q_e = q(e)
            z_e = z[e]
            if q_e > (1 + EPSILON) * z_e:
                delta = ln(
                    (q_e * (1 - (1 + EPSILON / 2) * z_e))
                    / ((1 - q_e) * (1 + EPSILON / 2) * z_e)
                )
                gamma[e] -= delta
                # Check that delta had the desired effect
                new_q_e = q(e)
                desired_q_e = (1 + EPSILON / 2) * z_e
                if round(new_q_e, 8) != round(desired_q_e, 8):
                    raise nx.NetworkXError(
                        f"Unable to modify probability for edge ({u}, {v})"
                    )
            else:
                in_range_count += 1
        # Check if the for loop terminated without changing any gamma
        if in_range_count == len(gamma):
            break

    # Remove the new edge attributes
    for _, _, d in G.edges(data=True):
        if lambda_key in d:
            del d[lambda_key]

    return gamma

In [ ]:
z_support = nx.MultiGraph()
for u, v in z_star:
    if (u, v) not in z_support.edges:
        edge_weight = min(G[u][v]['weight'], G[v][u]['weight'])
        z_support.add_edge(u, v, weight=edge_weight)


print(f"==================== printing z_support =====================")
for u,v in z_support.edges():
    print(min(u,v), max(u,v), z_support[u][v][0]['weight'])

print(f"==================== printing z_star (dict) =====================")
for u,v in z_star:
    if u >= v: continue
    print(u,v, z_star[(u,v)])


gamma = spanning_tree_distribution(z_support, z_star)
print(gamma)

==================== printing z_support =====================
0 2 5
0 11 0
0 13 5
2 9 3
2 10 3
2 13 0
5 11 8
6 11 8
9 13 3
1 10 0
1 12 0
9 10 0
7 12 5
9 12 0
8 9 5
9 16 5
3 4 0
3 6 6
3 8 12
4 5 6
4 7 12
4 8 12
6 14 0
7 8 0
8 16 0
5 15 0
7 16 0
14 15 0
==================== printing z_star (dict) =====================
0 2 0.23529411764705882
0 11 0.9411764705882353
0 13 0.7058823529411764
1 10 0.9411764705882353
1 12 0.9411764705882353
2 9 0.23529411764705882
2 10 0.47058823529411764
2 13 0.9411764705882353
3 4 0.9411764705882353
3 6 0.7058823529411764
3 8 0.23529411764705882
4 5 0.23529411764705882
4 7 0.23529411764705882
4 8 0.47058823529411764
5 11 0.7058823529411764
5 15 0.9411764705882353
6 11 0.23529411764705882
6 14 0.9411764705882353
7 8 0.23529411764705882
7 12 0.47058823529411764
7 16 0.9411764705882353
8 9 0.23529411764705882
8 16 0.7058823529411764
9 10 0.47058823529411764
9 12 0.47058823529411764
9 13 0.23529411764705882
9 16 0.23529411764705882
14 15 0.9411764705882353
{(0,

In [ ]:
from math import ceil, exp
from math import log as ln
import math

z_support = nx.Graph(z_support)
lambda_dict = {(u, v): exp(gamma[(u, v)]) for u, v in z_support.edges()}
nx.set_edge_attributes(z_support, lambda_dict, "weight")
# del gamma, lambda_dict

# Sample 2 * ceil( ln(n) ) spanning trees and record the minimum one
minimum_sampled_tree = None
minimum_sampled_tree_weight = math.inf
for _ in range(2 * ceil(ln(G.number_of_nodes()))):
    sampled_tree = random_spanning_tree(z_support, "weight", seed=None)
    sampled_tree_weight = sampled_tree.size(weight)
    if sampled_tree_weight < minimum_sampled_tree_weight:
        minimum_sampled_tree = sampled_tree.copy()
        minimum_sampled_tree_weight = sampled_tree_weight

# Orient the edges in that tree to keep the cost of the tree the same.
t_star = nx.MultiDiGraph()
for u, v, d in minimum_sampled_tree.edges(data=weight):
    if d == G[u][v]['weight']:
        t_star.add_edge(u, v, weight=d)
    else:
        t_star.add_edge(v, u, weight=d)

# Find the node demands needed to neutralize the flow of t_star in G
node_demands = {n: t_star.out_degree(n) - t_star.in_degree(n) for n in t_star}
nx.set_node_attributes(G, node_demands, "demand")

# Find the min_cost_flow
flow_dict = nx.min_cost_flow(G, "demand")

# Build the flow into t_star
for source, values in flow_dict.items():
    for target in values:
        if (source, target) not in t_star.edges and values[target] > 0:
            # IF values[target] > 0 we have to add that many edges
            for _ in range(values[target]):
                t_star.add_edge(source, target)

# Return the shortcut eulerian circuit
circuit = nx.eulerian_circuit(t_star, source=source)





def _shortcutting(circuit):
    """Remove duplicate nodes in the path"""
    nodes = []
    for u, v in circuit:
        if v in nodes:
            continue
        if not nodes:
            nodes.append(u)
        nodes.append(v)
    nodes.append(nodes[0])
    return nodes


    
ans = _shortcutting(circuit)


total_cost = 0
for i in range(len(ans)-1):
    total_cost += G[ans[i]][ans[i+1]]['weight']

print(f"total cost: {total_cost}")
print(ans)


total cost: 83
[16, 9, 2, 0, 11, 5, 6, 3, 14, 15, 4, 7, 12, 1, 10, 13, 8, 16]
